In [148]:
import pandas as pd
import subprocess
import numpy as np
from pprint import pprint
import json
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from src.config.configuration_manager import ConfigurationManager
from dagshub import get_repo_bucket_client
s3 = get_repo_bucket_client("Raj-Narayanan-B/scania_truck")
import os
import shutil
import glob
os.chdir(r"f:\\iNeuron\\Projects\\scania_failures_2")

Client created. Use the name of the repo (scania_truck) as the name of the bucket

[2024-02-14 19:56:41,300: INFO: helpers: Client created. Use the name of the repo (scania_truck) as the name of the bucket]


In [ ]:
s3.upload_file(
    Bucket="scania_truck",  # name of the repo
    Filename=r"F:\iNeuron\Projects\Data\Scania Truck Failures\aps_failure_training_set.csv",  # local path of file to upload
    Key="remote_aps_failure_training_set.csv",  # remote path where to upload the file
)

s3.upload_file(
    Bucket="scania_truck",  # name of the repo
    Filename=r"F:\iNeuron\Projects\Data\Scania Truck Failures\aps_failure_test_set.csv",  # local path of file to upload
    Key="remote_aps_failure_testing_set.csv",  # remote path where to upload the file
)

In [ ]:
s3.download_file(
    Bucket="scania_truck",  # name of the repo
    Key="remote_aps_failure_training_set.csv",  #  remote path from where to download the file
    Filename=r"F:\iNeuron\Projects\scania_failures_2\notebooks\sample_train.csv",  # local path where to download the file
)

s3.download_file(
    Bucket="scania_truck",  # name of the repo
    Key="remote_aps_failure_testing_set.csv",  #  remote path from where to download the file
    Filename=r"F:\iNeuron\Projects\scania_failures_2\notebooks\sample_test.csv",  # local path where to download the file
)


In [ ]:
filepath,file = os.path.split("artifacts/data/temp/test_data_3.csv")

In [ ]:
df = pd.read_csv(r'F:\iNeuron\Projects\scania_failures_2\notebooks\sample_train.csv')
df.head()

In [ ]:
train_df_1 = df.iloc[:,:74]
train_df_1['ident_id'] = range(1,60001)
train_df_1

In [ ]:
train_df_2 = df.iloc[:,74:148]
train_df_2['ident_id'] = range(1,60001)
train_df_2

In [ ]:
train_df_3 = df.iloc[:,148:]
train_df_3['ident_id'] = range(1,60001)
train_df_3

In [ ]:
df_test = pd.read_csv(r'F:\iNeuron\Projects\scania_failures_2\notebooks\sample_test.csv')
df_test.head()

In [ ]:
test_df_1 = df_test.iloc[:,:74]
test_df_1['ident_id'] = range(1,16001)
test_df_1

In [ ]:
test_df_2 = df_test.iloc[:,74:148]
test_df_2['ident_id'] = range(1,16001)
test_df_2

In [ ]:
test_df_3 = df_test.iloc[:,148:]
test_df_3['ident_id'] = range(1,16001)
test_df_3

In [ ]:
with open(r'F:\iNeuron\Projects\scania_failures_2\Secrets\Tokens\sample-token.json') as f:
        secrets = json.load(f)
        # logger.info(f"{config[1]} json file is loaded")

cloud_config = {'secure_connect_bundle': r'F:\iNeuron\Projects\scania_failures_2\Secrets\Bundles\secure-connect-sample.zip',
                'connect_timeout': None}
# print(f"cloud_config: {cloud_config}")
CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]
auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
# profile = ExecutionProfile(request_timeout=None)
cluster = Cluster(cloud=cloud_config,
                    auth_provider=auth_provider,
                    # execution_profiles={EXEC_PROFILE_DEFAULT: profile},
                    protocol_version=4)

session = cluster.connect()

In [ ]:
data_df = train_df_1
# Define keyspace name and table name
keyspace = 'sample_keyspace'
table_name = 'uploaded_data'
colums_types_dict_from_data_df = dict(zip(list(data_df.columns),list(data_df.dtypes.replace({'object':'text','int64':'int'}).values)))
# columns = ', '.join(train_df_1.columns)
columns = ', '.join([f"{key} {value}" for key,value in colums_types_dict_from_data_df.items()])
create_table_statement = f"CREATE TABLE IF NOT EXISTS {keyspace}.{table_name} ({columns}, PRIMARY KEY (ident_id));"
# create_table_statement = f"CREATE TABLE IF NOT EXISTS {keyspace}.{table_name} ({columns}, PRIMARY KEY (your_primary_key_column));"
session.execute(create_table_statement)


In [ ]:
dsbulk_command = [
    "dsbulk",
    "load",
    "-url", os.path.join("F:/iNeuron/Projects/scania_failures_2/artifacts/data/temp/","train_data1.csv"),
    "-k", "sample_keyspace",
    "-t", "uploaded_data",
    "-b", os.path.join("F:/iNeuron/Projects/scania_failures_2/Secrets/Bundles/","secure-connect-sample.zip"),
    "-u", "DfYsAxTkqZZWHwNguQHPBJMt", 
    "-p", "gl5rR3g-,_vTiPLZ.Zn7pouaWdAbZQ3elybJmlnaa,D+Zr4TLx4hpYNZrQwFHTARO6GXyeByF3BQS87plEhoSDPN50e,mZgbjqUm2IUTg5p3SojZF7r-GvA0gyNpbIdG",
]
result = subprocess.run(dsbulk_command, capture_output=True, text=True,shell=True)

print("Return code:", result.returncode)
print("Standard output:", result.stdout)
print("Standard error:", result.stderr)

In [239]:
train_df_1 = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\raw\train_data_1.csv")
train_df_1_ = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\temp\train_data_1.csv")

train_df_2 = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\raw\train_data_2.csv")
train_df_2_ = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\temp\train_data_2.csv")

train_df_3 = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\raw\train_data_3.csv")
train_df_3_ = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\temp\train_data_3.csv")

##################################################################################################################

test_df_1 = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\raw\test_data_1.csv")
test_df_1_ = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\temp\test_data_1.csv")

test_df_2 = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\raw\test_data_2.csv")
test_df_2_ = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\temp\test_data_2.csv")

test_df_3 = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\raw\test_data_3.csv")
test_df_3_ = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\temp\test_data_3.csv")

In [238]:
def checker_2(df_1:pd.DataFrame,df_2_:pd.DataFrame):
    df_1.sort_values(by = 'ident_id',inplace = True)
    df_1.reset_index(drop=True,inplace=True)
    df_1.drop(columns=['ident_id'],inplace=True)
    df_2_.drop(columns=['ident_id'],inplace=True)
    try:
        df_1.drop(columns=['field_74_'],inplace=True)
        df_2_.drop(columns=['class'],inplace=True)
    except:
        pass
    return (df_2_.compare(df_1))

In [245]:
checker_2(df_1 = train_df_1, df_2_=train_df_1_)

Empty DataFrame
Columns: []
Index: []

In [225]:
train_df_2.compare(train_df_1)

Index(['bm_000', 'bn_000', 'bo_000', 'bp_000', 'bq_000', 'br_000', 'bs_000',
       'bt_000', 'bu_000', 'bv_000', 'bx_000', 'by_000', 'bz_000', 'ca_000',
       'cb_000', 'cc_000', 'cd_000', 'ce_000', 'cf_000', 'cg_000', 'ch_000',
       'ci_000', 'cj_000', 'ck_000', 'cl_000', 'cm_000', 'cn_000', 'cn_001',
       'cn_002', 'cn_003', 'cn_004', 'cn_005', 'cn_006', 'cn_007', 'cn_008',
       'cn_009', 'co_000', 'cp_000', 'cq_000', 'cr_000', 'cs_000', 'cs_001',
       'cs_002', 'cs_003', 'cs_004', 'cs_005', 'cs_006', 'cs_007', 'cs_008',
       'cs_009', 'ct_000', 'cu_000', 'cv_000', 'cx_000', 'cy_000', 'cz_000',
       'da_000', 'db_000', 'dc_000', 'dd_000', 'de_000', 'df_000', 'dg_000',
       'dh_000', 'di_000', 'dj_000', 'dk_000', 'dl_000', 'dm_000', 'dn_000',
       'do_000', 'dp_000', 'dq_000', 'dr_000'],
      dtype='object')

In [226]:
train_df_2_.columns

Index(['bm_000', 'bn_000', 'bo_000', 'bp_000', 'bq_000', 'br_000', 'bs_000',
       'bt_000', 'bu_000', 'bv_000', 'bx_000', 'by_000', 'bz_000', 'ca_000',
       'cb_000', 'cc_000', 'cd_000', 'ce_000', 'cf_000', 'cg_000', 'ch_000',
       'ci_000', 'cj_000', 'ck_000', 'cl_000', 'cm_000', 'cn_000', 'cn_001',
       'cn_002', 'cn_003', 'cn_004', 'cn_005', 'cn_006', 'cn_007', 'cn_008',
       'cn_009', 'co_000', 'cp_000', 'cq_000', 'cr_000', 'cs_000', 'cs_001',
       'cs_002', 'cs_003', 'cs_004', 'cs_005', 'cs_006', 'cs_007', 'cs_008',
       'cs_009', 'ct_000', 'cu_000', 'cv_000', 'cx_000', 'cy_000', 'cz_000',
       'da_000', 'db_000', 'dc_000', 'dd_000', 'de_000', 'df_000', 'dg_000',
       'dh_000', 'di_000', 'dj_000', 'dk_000', 'dl_000', 'dm_000', 'dn_000',
       'do_000', 'dp_000', 'dq_000', 'dr_000'],
      dtype='object')

In [180]:
def process(df:pd.DataFrame):
    df.replace('na', np.nan, inplace=True)
    df.drop(columns = ['ident_id'], inplace=True)     
    df.rename(columns={'field_74_': 'class'}, inplace=True)  
    for i in df.columns:
        if i != 'class':
            df[i] = df[i].astype('float')
        else:
            df['class'] = df['class'].map({'neg':0,'pos':1})
            df.drop(columns = ['class'], inplace=True)   
    return(df)

def describe_check(df_1:pd.DataFrame, df_2:pd.DataFrame):
    checks = {}
    stats_cols = ['count','mean','std','min','25%','50%','75%','max']
    for i in stats_cols:
        checks[i] = all(df_1.describe().T[i] == df_2.describe().T[i])
    return checks
    

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,bd_000,be_000,bf_000,bg_000,bh_000,bi_000,bj_000,bk_000,bl_000,ident_id
0,neg,76698,na,2130706438,280,0,0,0,0,0,...,10,108,50,2551696,97518,947550,799478,330760,353400,1
1,neg,33058,na,0,na,0,0,0,0,0,...,454,364,350,1393352,49028,688314,392208,341420,359780,2
2,neg,41040,na,228,100,0,0,0,0,0,...,202,576,4,1234132,28804,160176,139730,137160,130640,3
3,neg,12,0,70,66,0,10,0,0,0,...,12,0,0,2668,184,7632,3090,na,na,4
4,neg,60874,na,1368,458,0,0,0,0,0,...,556,642,2,1974038,86454,653692,399410,306780,282560,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,neg,153002,na,664,186,0,0,0,0,0,...,1206,1606,14,4880368,129760,1244058,678792,263320,221620,59996
59996,neg,2286,na,2130706538,224,0,0,0,0,0,...,14,716,0,56982,1900,14404,9572,na,na,59997
59997,neg,112,0,2130706432,18,0,0,0,0,0,...,10,2,0,8784,480,7640,6224,na,na,59998
59998,neg,80292,na,2130706432,494,0,0,0,0,0,...,78,424,70,2634394,105862,580084,1032454,302700,332300,59999


True

In [151]:
train_df_1_transformed = process(train_df_1)
train_df_1__transformed = process(train_df_1_)

describe_check(train_df_1_transformed,train_df_1__transformed)

{'count': True,
 'mean': True,
 'std': False,
 'min': True,
 '25%': True,
 '50%': True,
 '75%': True,
 'max': True}

In [177]:
train_df_1_transformed

,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,bc_000,bd_000,be_000,bf_000,bg_000,bh_000,bi_000,bj_000,bk_000,bl_000
0,2184.0,NaN,2.160000e+02,140.0,0.0,0.0,0.0,0.0,0.0,0.0,...,40.0,40.0,46.0,0.0,83702.0,4020.0,31144.0,31022.0,NaN,NaN
1,124.0,NaN,8.000000e+00,6.0,0.0,0.0,0.0,0.0,0.0,506.0,...,2.0,16.0,130.0,0.0,6366.0,348.0,8060.0,3386.0,1310700.0,1310700.0
2,1084202.0,NaN,NaN,NaN,0.0,0.0,0.0,3808.0,419386.0,5982232.0,...,NaN,NaN,57326.0,0.0,20994370.0,623010.0,7717282.0,6533948.0,240060.0,240660.0
3,356.0,NaN,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,132.0,48.0,0.0,12872.0,604.0,2808.0,4496.0,NaN,NaN
4,194.0,4.0,3.200000e+01,10.0,0.0,0.0,0.0,0.0,0.0,1974.0,...,18.0,12.0,44.0,0.0,43820.0,1142.0,17586.0,16034.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,411606.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59996,4.0,NaN,2.000000e+00,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,1946.0,166.0,3456.0,1310.0,1310700.0,1310700.0
59997,802.0,NaN,1.000000e+01,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,72.0,16.0,0.0,19510.0,342.0,1532.0,2952.0,NaN,NaN
59998,1474.0,0.0,2.130706e+09,26.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,6.0,4.0,0.0,43916.0,768.0,7062.0,8818.0,NaN,NaN


In [178]:
train_df_1__transformed

,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,bc_000,bd_000,be_000,bf_000,bg_000,bh_000,bi_000,bj_000,bk_000,bl_000
0,76698.0,NaN,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,108.0,50.0,2551696.0,97518.0,947550.0,799478.0,330760.0,353400.0
1,33058.0,NaN,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,86.0,454.0,364.0,350.0,1393352.0,49028.0,688314.0,392208.0,341420.0,359780.0
2,41040.0,NaN,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,128.0,202.0,576.0,4.0,1234132.0,28804.0,160176.0,139730.0,137160.0,130640.0
3,12.0,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,318.0,...,2.0,12.0,0.0,0.0,2668.0,184.0,7632.0,3090.0,NaN,NaN
4,60874.0,NaN,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,0.0,...,448.0,556.0,642.0,2.0,1974038.0,86454.0,653692.0,399410.0,306780.0,282560.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,153002.0,NaN,6.640000e+02,186.0,0.0,0.0,0.0,0.0,0.0,2564.0,...,1304.0,1206.0,1606.0,14.0,4880368.0,129760.0,1244058.0,678792.0,263320.0,221620.0
59996,2286.0,NaN,2.130707e+09,224.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,14.0,716.0,0.0,56982.0,1900.0,14404.0,9572.0,NaN,NaN
59997,112.0,0.0,2.130706e+09,18.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,10.0,2.0,0.0,8784.0,480.0,7640.0,6224.0,NaN,NaN
59998,80292.0,NaN,2.130706e+09,494.0,0.0,0.0,0.0,0.0,0.0,0.0,...,64.0,78.0,424.0,70.0,2634394.0,105862.0,580084.0,1032454.0,302700.0,332300.0


In [152]:
train_df_2_transformed = process(train_df_2)
train_df_2__transformed = process(train_df_2_)

describe_check(train_df_2_transformed,train_df_2__transformed)

{'count': True,
 'mean': False,
 'std': False,
 'min': True,
 '25%': True,
 '50%': True,
 '75%': True,
 'max': True}

In [153]:
train_df_3_transformed = process(train_df_3)
train_df_3__transformed = process(train_df_3_)

describe_check(train_df_3_transformed,train_df_3__transformed)

{'count': True,
 'mean': True,
 'std': False,
 'min': True,
 '25%': True,
 '50%': True,
 '75%': True,
 'max': True}

In [154]:
test_df_1_transformed = process(test_df_1)
test_df_1__transformed = process(test_df_1_)

describe_check(test_df_1_transformed,test_df_1__transformed)

{'count': True,
 'mean': True,
 'std': False,
 'min': True,
 '25%': True,
 '50%': True,
 '75%': True,
 'max': True}

In [155]:
test_df_2_transformed = process(test_df_2)
test_df_2__transformed = process(test_df_2_)

describe_check(test_df_2_transformed,test_df_2__transformed)

{'count': True,
 'mean': False,
 'std': False,
 'min': True,
 '25%': True,
 '50%': True,
 '75%': True,
 'max': True}

In [156]:
test_df_3_transformed = process(test_df_3)
test_df_3__transformed = process(test_df_3_)

describe_check(test_df_3_transformed,test_df_3__transformed)

{'count': True,
 'mean': False,
 'std': False,
 'min': True,
 '25%': True,
 '50%': True,
 '75%': True,
 'max': True}

In [162]:
checker = train_df_1_transformed.describe().T['std'] != train_df_1__transformed.describe().T['std']

In [174]:
train_df_1_transformed.describe().T['std'][checker]

af_000    False
ag_001    False
ai_000    False
aj_000    False
ak_000    False
al_000    False
an_000    False
as_000    False
at_000    False
ay_000    False
ay_001    False
ay_002    False
ay_003    False
ay_006    False
ay_007    False
ay_009    False
az_000    False
az_004    False
az_009    False
ba_000    False
ba_002    False
bb_000    False
bc_000    False
bi_000    False
Name: std, dtype: bool

In [164]:
train_df_1__transformed.describe().T['std'][checker]

af_000    2.097926e+02
ag_001    3.420053e+04
ai_000    1.632778e+05
aj_000    5.035971e+04
ak_000    7.583162e+04
al_000    5.394658e+05
an_000    7.790350e+06
as_000    1.101004e+04
at_000    1.196159e+05
ay_000    4.544963e+05
ay_001    5.352707e+05
ay_002    4.283370e+05
ay_003    2.064679e+05
ay_006    3.278941e+06
ay_007    5.106177e+06
ay_009    9.796080e+04
az_000    7.363676e+04
az_004    4.184087e+06
az_009    3.256570e+03
ba_000    3.777091e+06
ba_002    1.196083e+06
bb_000    1.088674e+07
bc_000    4.045937e+03
bi_000    1.485185e+06
Name: std, dtype: float64

In [165]:
2.097926e+02

209.7926

In [166]:
2.097926e+02

209.7926

In [167]:
checker_mean = train_df_2_transformed.describe().T['mean'] != train_df_2__transformed.describe().T['mean']

In [168]:
train_df_2_transformed.describe().T['mean'][checker_mean]

ci_000    3.481204e+06
cj_000    1.028419e+05
ck_000    7.143427e+05
Name: mean, dtype: float64

In [169]:
train_df_2__transformed.describe().T['mean'][checker_mean]

ci_000    3.481204e+06
cj_000    1.028419e+05
ck_000    7.143427e+05
Name: mean, dtype: float64

In [170]:
7.143427e+05

714342.7

In [171]:
7.143427e+05


714342.7

In [ ]:
checker_std = train_df_2_transformed.describe().T['std'] != train_df_2__transformed.describe().T['std']

In [ ]:
train_df_2_transformed.describe().T['std'][checker_std]

In [ ]:
train_df_2__transformed.describe().T['std'][checker_std]

In [ ]:
8.355997e+06


In [ ]:
8.355997e+06


In [ ]:
test_checker_mean = test_df_2_transformed.describe().T['mean'] != test_df_2__transformed.describe().T['mean']

In [ ]:
test_df_2_transformed.describe().T['mean'][test_checker_mean]

In [ ]:
test_df_2__transformed.describe().T['mean'][test_checker_mean]

In [ ]:
3.656347e+06

In [ ]:
3.656347e+06

In [ ]:
test_checker_std = test_df_2_transformed.describe().T['std'] != test_df_2__transformed.describe().T['std']

In [ ]:
test_df_2_transformed.describe().T['std'][test_checker_std]

In [ ]:
test_df_2__transformed.describe().T['std'][test_checker_std]

In [ ]:
8.775294e+06


In [ ]:
8.775294e+06


In [142]:
directory_path = "logs"
pattern = os.path.join(directory_path, "LOAD*")
pattern

'logs\\LOAD*'

In [143]:
# Use glob to get a list of file paths matching the pattern
files_to_remove = glob.glob(pattern)
files_to_remove

['logs\\LOAD_20240214-124335-267000',
 'logs\\LOAD_20240214-124600-189000',
 'logs\\LOAD_20240214-124640-477000',
 'logs\\LOAD_20240214-131615-512000',
 'logs\\LOAD_20240214-131654-888000',
 'logs\\LOAD_20240214-132218-146000',
 'logs\\LOAD_20240214-132637-497000',
 'logs\\LOAD_20240214-134512-002000',
 'logs\\LOAD_20240214-134550-974000',
 'logs\\LOAD_20240214-140204-085000']

In [147]:
# Iterate through the files and remove them
for file_path in files_to_remove:
    shutil.rmtree(file_path)
    print(f"Removed: {file_path}")

print("Files removed successfully.")

Removed: logs\LOAD_20240214-124335-267000
Removed: logs\LOAD_20240214-124600-189000
Removed: logs\LOAD_20240214-124640-477000
Removed: logs\LOAD_20240214-131615-512000
Removed: logs\LOAD_20240214-131654-888000
Removed: logs\LOAD_20240214-132218-146000
Removed: logs\LOAD_20240214-132637-497000
Removed: logs\LOAD_20240214-134512-002000
Removed: logs\LOAD_20240214-134550-974000
Removed: logs\LOAD_20240214-140204-085000
Files removed successfully.


In [145]:
pwd

'f:\\iNeuron\\Projects\\scania_failures_2'